# **강화학습 중간고사 시험 문제**
- 코드의 단계별로 작성하고 설명 작성하세요.
- 각각의 ipynb 파일로 제출하세요.
- 파일명 예시: **202500000 홍길동_Jack's Car rental.ipynb**

# 학번과 이름을 기입해주세요. (한글)

- 학번: 202192022
- 이름: 여명구

## **2. Jack's Car rental**

### 문제:
- 잭(Jack)은 두 곳의 렌트카 지점(A, B)을 운영하고 있음.
- 매일 밤 두 지점 간에 차량을 재배치하여 다음 날의 렌트 수익의 최대화하고 싶어함.
- **수익을 최대화하기 위해서 잭(Jack)은 어떻게 해야 하는가?**

### 조건
- 수익: 차량 1대를 렌트하면 $10의 수익이 발생합니다.

- 비용: 지점 간 차량 1대를 이동시키면 $2의 비용이 발생합니다.

- 제약
  1) 각 지점은 최대 20대의 차량만 보유할 수 있습니다. (초과분은 사라짐)
  2) 밤에 이동시킬 수 있는 차량은 최대 5대입니다.


※ 제출 내용
  1)  최적 정책 (시각화)
  2)  최적 가치 함수 (시각화)

In [1]:
'''
이 코드는 '잭의 렌터카(Jack's Car Rental)' 문제를 해결하기 위해
'정책 반복(Policy Iteration)' 알고리즘을 구현합니다.

이 문제는 PDF 자료에서 제시된 MDP(S, A, P, R) 정의를 따릅니다.
- S (상태): (A지점 차량 수, B지점 차량 수) [cite: 189]
- A (행동): A->B로 이동하는 차량 수 (-5 ~ +5) [cite: 192]
- P (전이): 수요/반납이 포아송 분포를 따름 [cite: 152, 202, 203]
- R (보상): 대여료($10) - 이동비용($2) [cite: 148]

알고리즘은 [정책 평가]와 [정책 개선]을 수렴할 때까지 반복합니다[cite: 221].
'''

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # 시각화를 위한 Seaborn 사용
from scipy.stats import poisson

# --- 1. 환경 및 문제 조건 설정 ---

# 제약 조건
MAX_CARS = 20           # 각 지점 최대 차량 수 [cite: 143]
MAX_MOVE = 5            # 밤 사이 최대 이동 가능 차량 수 [cite: 145]
STATE_SPACE = MAX_CARS + 1 # (0 ~ 20, 총 21개 상태)

# 보상 및 비용
RENTAL_PROFIT = 10      # 대여 수익 [cite: 148]
MOVE_COST = 2           # 이동 비용

# MDP 파라미터
GAMMA = 0.9             # 할인율 (PDF 예시) [cite: 217]

# PDF에서 가정한 포아송 분포 파라미터 [cite: 153, 154]
LAMBDA_REQ_1 = 3  # 1번 지점(A) 평균 요청
LAMBDA_RET_1 = 3  # 1번 지점(A) 평균 반납
LAMBDA_REQ_2 = 4  # 2번 지점(B) 평균 요청
LAMBDA_RET_2 = 2  # 2번 지점(B) 평균 반납

# 계산 최적화: 포아송 분포는 꼬리가 길기 때문에
# 확률이 매우 낮은 (e.g., 11대 이상) 경우는 계산에서 제외 [cite: 211]
MAX_POISSON_K = 11      
# 포아송 확률값 캐시 (매번 계산하는 것을 방지)
poisson_cache = dict()

def get_poisson_prob(k, lam):
    """
    k와 lambda에 대한 포아송 확률(PMF)을 계산합니다.
    (계산된 값을 캐시에 저장하여 중복 계산 방지)
    """
    if (k, lam) not in poisson_cache:
        poisson_cache[(k, lam)] = poisson.pmf(k, lam)
    return poisson_cache[(k, lam)]

# --- 2. 핵심 함수: 기대 보상 계산 (4중 루프) ---

def calculate_expected_return(state, action, V):
    """
    현재 상태(s), 행동(a), 현재 가치함수(V)를 바탕으로
    벨만 방정식의 우변(기대 보상 + 기대 가치)을 계산합니다.
    이것이 이 문제의 가장 복잡한 '전이(P)'와 '보상(R)' 부분입니다.
    """
    
    # 1. 즉각적인 보상(비용)
    # 행동(action)은 1->2로 옮기는 차의 수.
    expected_return = -MOVE_COST * abs(action)
    
    # 2. 행동(a) 적용 후 아침 차량 대수
    # (제약 조건 1: 최대 20대)
    n1_start = min(state[0] - action, MAX_CARS)
    n2_start = min(state[1] + action, MAX_CARS)

    # 3. 모든 확률적 사건(수요/반납)에 대한 기댓값 계산 (4중 루프)
    for req1 in range(MAX_POISSON_K):
        for req2 in range(MAX_POISSON_K):
            for ret1 in range(MAX_POISSON_K):
                for ret2 in range(MAX_POISSON_K):
                    
                    # 4. 이 사건이 발생할 확률 (P)
                    # 모든 사건은 독립적이므로 확률을 곱함 [cite: 204]
                    prob = (
                        get_poisson_prob(req1, LAMBDA_REQ_1) *
                        get_poisson_prob(req2, LAMBDA_REQ_2) *
                        get_poisson_prob(ret1, LAMBDA_RET_1) *
                        get_poisson_prob(ret2, LAMBDA_RET_2)
                    )
                    
                    # 5. 실제 렌트된 차량 수 (수요가 재고보다 많을 수 없음)
                    rented1 = min(n1_start, req1)
                    rented2 = min(n2_start, req2)
                    
                    # 6. 확률적 보상 (R)
                    profit = (rented1 + rented2) * RENTAL_PROFIT
                    
                    # 7. 하루 종료 시 차량 수 (다음 상태 s')
                    # (제약 조건 1: 최대 20대) [cite: 207]
                    n1_end = min(n1_start - rented1 + ret1, MAX_CARS)
                    n2_end = min(n2_start - rented2 + ret2, MAX_CARS)
                    
                    # 8. E[R + γV(s')] 누적
                    # (V[n1_end, n2_end]는 다음 상태의 가치)
                    expected_return += prob * (profit + GAMMA * V[n1_end, n2_end])

    return expected_return

# --- 3. 알고리즘: 정책 반복 (Policy Iteration) ---

def policy_iteration():
    """
    정책 반복(Policy Iteration)을 수행하여
    최적 정책(Policy)과 최적 가치(V)를 찾습니다.
    """
    
    # 1. 초기화
    # V: 21x21 크기, 0으로 초기화
    V = np.zeros((STATE_SPACE, STATE_SPACE)) 
    # Policy: 21x21 크기, 0으로 초기화 (행동: "이동 안 함")
    Policy = np.zeros((STATE_SPACE, STATE_SPACE), dtype=int) 
    
    iteration = 0
    while True:
        iteration += 1
        print(f"\n===== [정책 반복 Iteration {iteration}] =====")
        
        # --- [STEP 1: 정책 평가 (Policy Evaluation)] ---
        # 현재 정책(Policy)이 얼마나 좋은지, V가 수렴할 때까지 계산 [cite: 222]
        print("1. 정책 평가 (Policy Evaluation) 시작...")
        while True:
            delta = 0
            # V(k+1) 계산을 위해 V(k)를 복사
            V_old = V.copy() 
            
            # 441개 모든 상태(s)에 대해
            for n1 in range(STATE_SPACE):
                for n2 in range(STATE_SPACE):
                    state = (n1, n2)
                    # 현재 정책(Policy)이 정해준 행동(a)
                    action = Policy[state] 
                    
                    # V(k+1)[s] <- E[R + γV(k)[s']]
                    V[state] = calculate_expected_return(state, action, V_old)
                    
                    delta = max(delta, abs(V[state] - V_old[state]))
            
            print(f"   V 가치 최대 변화량(delta): {delta:.4f}")
            # 가치 함수가 수렴하면 평가 종료
            if delta < 1e-4: # (수렴 기준값)
                print("   정책 평가 완료. V 수렴.")
                break

        # --- [STEP 2: 정책 개선 (Policy Improvement)] ---
        # 계산된 V를 바탕으로 더 나은 행동이 있는지 탐색 [cite: 223]
        print("2. 정책 개선 (Policy Improvement) 시작...")
        policy_stable = True
        
        for n1 in range(STATE_SPACE):
            for n2 in range(STATE_SPACE):
                state = (n1, n2)
                old_action = Policy[state]
                
                action_returns = []
                
                # 제약 2)를 고려한 가능한 모든 행동 (-5 ~ +5)
                # (단, 현재 재고를 초과할 수 없음)
                valid_actions = range(
                    max(-MAX_MOVE, -n2),               # B에서 5대 or n2대 이상 못 뺌
                    min(MAX_MOVE, n1) + 1              # A에서 5대 or n1대 이상 못 뺌
                )
                
                # 모든 가능한 행동 'a'에 대해
                for action in valid_actions:
                    # Q(s, a) = E[R + γV(s')] 계산
                    q_value = calculate_expected_return(state, action, V)
                    action_returns.append(q_value)
                
                # 가장 좋은(Greedy) 행동을 새 정책으로 선택
                new_action = valid_actions[np.argmax(action_returns)]
                Policy[state] = new_action
                
                # 정책이 하나라도 바뀌었는지 확인
                if old_action != new_action:
                    policy_stable = False
        
        if policy_stable:
            print(f"정책 개선 완료. 정책이 안정됨(Stable).")
            print(f"===== [최적 정책 발견! Iteration {iteration}에서 종료] =====")
            break # 메인 루프 종료
        else:
            print(f"정책 개선 완료. 정책이 변경됨(Unstable).")

    return Policy, V

# --- 4. 시각화 함수 ---

def plot_results(Policy, V):
    """
    최종 최적 정책(π*)과 최적 가치(V*)를 시각화합니다.
    """
    
    # 플롯 1: 최적 가치 함수 (V*) - 3D 표면 플롯
    # (PDF 12페이지의 V4 그래프와 유사 [cite: 179])
    fig = plt.figure(figsize=(12, 7))
    ax = fig.add_subplot(111, projection='3d')
    X = np.arange(0, STATE_SPACE) # Loc 2 (0~20)
    Y = np.arange(0, STATE_SPACE) # Loc 1 (0~20)
    X, Y = np.meshgrid(X, Y)
    
    # (V 배열의 (n1, n2) 축이 (Y, X)와 일치함)
    ax.plot_surface(X, Y, V, cmap='viridis')
    ax.set_title('Optimal Value Function (V*)')
    ax.set_xlabel('# Cars at Location 2')
    ax.set_ylabel('# Cars at Location 1')
    ax.set_zlabel('Expected Return ($)')
    plt.show()

    # 플롯 2: 최적 정책 (π*) - 2D 히트맵
    # (PDF 12페이지의 π4 그래프와 유사 [cite: 169])
    plt.figure(figsize=(10, 8))
    # (정책 배열을 뒤집어야 (0,0)이 좌하단에 옴)
    Policy_flipped = np.flipud(Policy) 
    
    ax = sns.heatmap(
        Policy_flipped, 
        cmap='coolwarm',  # (양수: A->B(파랑), 음수: B->A(빨강))
        center=0,         # 0을 흰색으로
        annot=True,       # 숫자 표시
        fmt='d',          # 정수로
        linewidths=.5
    )
    ax.set_title('Optimal Policy (π*) [Cars moved from Loc 1 to Loc 2]')
    ax.set_xlabel('# Cars at Location 2')
    ax.set_ylabel('# Cars at Location 1')
    # Y축 레이블을 (0, 5, ..., 20)으로 수정 (뒤집었기 때문)
    ax.set_yticklabels(
        np.arange(MAX_CARS, -1, -5), 
        rotation=0, 
        va='center'
    )
    ax.set_xticklabels(np.arange(0, STATE_SPACE, 5))
    plt.show()


# --- 5. 메인 실행 ---
if __name__ == "__main__":
    
    print("===== [잭의 렌터카 문제 풀이 시작] =====")
    print("알고리즘: 정책 반복 (Policy Iteration)")
    print(f"가정: γ={GAMMA}, λ(req1,ret1)=({LAMBDA_REQ_1},{LAMBDA_RET_1}), λ(req2,ret2)=({LAMBDA_REQ_2},{LAMBDA_RET_2})\n")
    
    # 1. 문제 풀이
    optimal_policy, optimal_value = policy_iteration()
    
    # 2. 결과 출력 (중요 요소 확인)
    print("\n\n--- [제출 1] 최적 정책 (π*) ---")
    print("(각 칸은 (A지점 차량, B지점 차량) 상태에서 A->B로 이동시킬 차량 수)")
    print(optimal_policy)
    
    print("\n\n--- [제출 2] 최적 가치 함수 (V*) ---")
    print("(각 칸은 해당 상태의 총 기대 수익)")
    print(optimal_value.round(1))
    
    # 3. 결과 시각화
    print("\n\n--- [시각화] 결과 플롯 생성 중 ---")
    plot_results(optimal_policy, optimal_value)
    
    print("===== [문제 풀이 완료] =====")

c:\Users\brigh\Documents\GitHub\warehouse\.venv\Lib\site-packages\seaborn\_statistics.py:32: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.3)
  from scipy.stats import gaussian_kde


===== [잭의 렌터카 문제 풀이 시작] =====
알고리즘: 정책 반복 (Policy Iteration)
가정: γ=0.9, λ(req1,ret1)=(3,3), λ(req2,ret2)=(4,2)


===== [정책 반복 Iteration 1] =====
1. 정책 평가 (Policy Evaluation) 시작...
   V 가치 최대 변화량(delta): 69.5240
   V 가치 최대 변화량(delta): 62.3569
   V 가치 최대 변화량(delta): 55.9161
   V 가치 최대 변화량(delta): 49.9777
   V 가치 최대 변화량(delta): 44.2147
   V 가치 최대 변화량(delta): 38.5643
   V 가치 최대 변화량(delta): 33.2485
   V 가치 최대 변화량(delta): 28.5103
   V 가치 최대 변화량(delta): 24.4605
   V 가치 최대 변화량(delta): 21.0821
   V 가치 최대 변화량(delta): 18.2882
   V 가치 최대 변화량(delta): 15.9723
   V 가치 최대 변화량(delta): 14.0354
   V 가치 최대 변화량(delta): 12.3964
   V 가치 최대 변화량(delta): 10.9925
   V 가치 최대 변화량(delta): 9.7771
   V 가치 최대 변화량(delta): 8.7153
   V 가치 최대 변화량(delta): 7.7813
   V 가치 최대 변화량(delta): 6.9552
   V 가치 최대 변화량(delta): 6.2219
   V 가치 최대 변화량(delta): 5.5691
   V 가치 최대 변화량(delta): 4.9869
   V 가치 최대 변화량(delta): 4.4668
   V 가치 최대 변화량(delta): 4.0018
   V 가치 최대 변화량(delta): 3.5858
   V 가치 최대 변화량(delta): 3.2135
   V 가치 최대 변화량(delta): 2.

KeyboardInterrupt: 